In [1]:
# !pip install datasets
# !pip install librosa

In [180]:
import IPython.display as pds
import numpy as np
import pandas as pd
import warnings
import random
import zipfile
import librosa
import os
import re
import json
import tensorflow as tf
warnings.filterwarnings("ignore")

from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from scipy import signal

In [3]:
!git clone https://github.com/Bangkit-Capstone-Team/IOH-Chat-App.git

fatal: destination path 'IOH-Chat-App' already exists and is not an empty directory.


In [4]:
SPEECH_DATA_DIR = "/content/IOH-Chat-App/Machine Learning/datasets/speech/audio.zip"

zf = zipfile.ZipFile(SPEECH_DATA_DIR)
zf.extractall("/tmp")
zf.close()

In [146]:
DATASET_PATH = "/tmp/audio/"

In [190]:
def decode_audio(audio_path):
  audio, _ = librosa.load(audio_path)
  return audio

def get_label(file_path):
  string_split = os.path.split(file_path)
  result = re.findall(r"^\w*", string_split[-1])[0]
  return result

def audio_signal_and_label(file_path):
  label = get_label(file_path)
  feature = decode_audio(file_path)
  return feature, label

def prepare_dataset(dataset_path):
  json_data = {
      "labels": list(),
      "features": list(),
      "one_hot_labels": list(),
  }

  filenames = os.listdir(dataset_path)
  audio_path = [os.path.join(dataset_path, filename) for filename in filenames]

  for i, path in enumerate(audio_path):
    if os.path.getsize(path) != 0:
      features, labels = audio_signal_and_label(path)

      json_data["labels"].append(labels)
      json_data["features"].append(features)
      json_data["one_hot_labels"].append(i)

      i =+ 1

  return json_data

In [ ]:
dataset = prepare_dataset(DATASET_PATH)

In [227]:
df = pd.DataFrame(dataset)
df.head()

,labels,features,one_hot_labels
0,100,"[[-565.8677368164062, 26.238544464111328, 22.8...",0
1,100000,"[[-583.0913696289062, 16.86783218383789, 14.79...",1
2,15,"[[-518.384765625, 29.100135803222656, 26.38721...",2
3,16,"[[-553.1329956054688, 0.0, 0.0, 0.0, 0.0, 0.0,...",3
4,17,"[[-541.1782836914062, 12.078079223632812, 11.7...",4


In [231]:
features = np.array(df.features)
labels = np.array(df.labels)

x_train, y_train, x_test, y_test = train_test_split(features, labels, test_size=0.2, random_state=1)

x_train = np.expand_dims(x_train, axis=1)
x_test = np.expand_dims(x_test, axis=1)

input_shape = (x_train.shape[0], x_train.shape[1], 1)

(1527, 1, 1)


In [235]:
def getModel(n_class, input_shape):
  model = tf.keras.Sequential()

  model.add(layers.Input(shape=input_shape))
  model.add(layers.Resizing(32, 32))
  
  model.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), padding="same"))

  model.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), padding="same"))
  
  model.add(layers.Conv2D(128, (3, 3), activation='relu', padding="same"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), padding="same"))
  
  model.add(layers.Flatten())

  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(n_class, activation='softmax'))

  return model

In [236]:
LR = 1e-4
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LR)
LOSS = tf.keras.losses.sparse_categorical_crossentropy

model = getModel(len(y_train), input_shape)

model.compile(optimizer=OPTIMIZER,
              loss=LOSS,
              metrics=["accuracy"])

model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_2 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 conv2d_35 (Conv2D)          (None, 32, 32, 32)        320       
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 8, 8, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_37 (Conv2D)          (None, 8, 8, 128)       

In [237]:
EPOCHS = 15

model.fit(x_train, 
          y_train, 
          epochs=EPOCHS, 
          validation_data=(x_test, y_test), 
          batch_size=128)

ValueError: ignored